# Nature of Work Mapping for New Data Set

In [1]:
import pandas as pd
import numpy as np
from google.cloud import storage
from google.cloud import bigquery
# Connection to Bigquery
client = bigquery.Client(project='prj-prod-dataplatform')


# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Settings in this Notebook
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", None)


In [2]:
sq = """with base as 
(select digitalLoanAccountId, nature_of_work, industry_description, employment_type, source_funds from `worktable_data_analysis.Demographic_Beta1_Jan2023_dec2024` lmt)
select * from base; """
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID b6ce06d8-94a4-4236-9f01-4c67d77ae1f7 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [3]:
df.shape

(1177784, 5)

In [4]:
df.groupby(['nature_of_work'])['digitalLoanAccountId'].nunique()

nature_of_work
Accountant/Auditor                                              19005
Admin/Clerical Staff/Officer                                    56980
Architect/Interior Decorator                                     2855
Bank Employee                                                   14844
Broker/Dealer                                                    2485
Call Center Agent/Tele Marketer                                115422
Cashier/Food Server/Waiter                                      35256
Consultant                                                       7058
Court Personnel/Bailiff                                           121
Debt Collection/Recovery Agents/ Collection Agency Employee      4163
Doctor/Dentist/Medical Professional                             17432
Engineer                                                        17085
Entertainer/Artist                                               3292
Farmer/Fisherman                                                 4967
Fire 

In [5]:
# Mapping dictionary
mapping = {
    'Admin/Clerical Staff/Officer': 'missing',
    'Architect/Interior Decorator': 'Licensed Professional - Others',
    'Bank Employee': 'missing',
    'Broker/Dealer': 'Broker/Dealer',
    'Call Center Agent/Tele Marketer': 'missing',
    'Cashier/Food Server/Waiter': 'missing',
    'Consultant': 'missing',
    'Doctor/Dentist/Medical Professional': 'Licensed Professional - Others',
    'Engineer': 'Licensed Professional - Others',
    'Entertainer/Artist': 'missing',
    'Foreign Ambassador/Diplomat': 'missing',
    'Govt Employee': 'Govt Employee',
    'Insurance Agent/Financial Advisor': 'missing',
    'IT Professional': 'missing',
    'Lawyer': 'Licensed Professional - Lawyer',
    'Military/Armed Forces/Police/Navy/Airforce': 'Government - Uniformed Service',
    'Newscaster/Media Personality': 'missing',
    'Pilot/Airline Crew': 'missing',
    'Seafarer': 'missing',
    'Teacher/Instructor/Coach': 'missing',
    'Utility Personnel/Household Help': 'missing',
    'Accountant/Auditor': 'Licensed Professional - Accountant ',
    'Utilities (Electricity, Gas and Water)': 'missing',
    'Wholesale and Retail Trade': 'missing',
    'Sales/Marketing Personnel': 'Sales/Marketing Personnel',
    'Other Professional services': 'Licensed Professional - Others',
    'Other Non Professional Services': 'missing',
    'Owner': 'Owner',
    'Religious Leader': 'Religious Leader',
    'Fire Fighter': 'Government - Uniformed Service',
    'Farmer/Fisherman': 'missing',
    'Debt Collection/Recovery Agents/ Collection Agency Employee': 'Debt Collection/Recovery Agents/ Collection Agency Employee',
    'Court Personnel/Bailiff': 'Court Personnel/Bailiff',
    'Store/Service Manager': 'Manager/Executive',
    'Sales Marketing Professional': 'Sales/Marketing Personnel',
    'Barber / Beauty Salon': 'Sales/Marketing Personnel',
    'Business Owner (MSME)': 'Owner',
    'Delivery Rider/Driver': 'Delivery Rider/Driver',
    'Manpower Agency': 'missing',
    'Online Freelancing / Virtual Assistant / Online teacher': 'Virtual/Online/Remote Employee',
    'Online Seller': 'Virtual/Online/Remote Employee',
    'Real Estate agent': 'missing',
    'Skilled Worker': 'Skilled Worker',
    'Technician (Cellphone / Laptop)': 'missing',
    'Vendor (Sari-Sari store, Wet market, etc)': 'missing',
    'Staff/Rank and File': 'Staff/Rank and File',
    'Officer/Supervisor ': 'Officer/Supervisor ',
    'Manager/Executive': 'Manager/Executive',
    'Licensed Professional - Lawyer': 'Licensed Professional - Lawyer',
    'Licensed Professional - Accountant ': 'Licensed Professional - Accountant ',
    'Licensed Professional - Others': 'Licensed Professional - Others',
    'Religious Minister ': 'Religious Minister ',
    'Government - Elected Official ': 'Government - Elected Official ',
    'Government - Uniformed Service': 'Government - Uniformed Service',
    'Director/Trustee/Stockholder': 'Director/Trustee/Stockholder',
    'Virtual/Online/Remote Employee': 'Virtual/Online/Remote Employee',
    'Government - Appointed Official': 'Government - Appointed Official'
}

# Apply the mapping
df['nature_of_work_new'] = df['nature_of_work'].map(mapping)

# Handle the special case for 'Govt Official (Appointed/Elected)'
def assign_govt_official():
    return np.random.choice(
        ['Government - Appointed Official', 'Government - Elected Official'],
        p=[0.9, 0.1]
    )

df.loc[df['nature_of_work'] == 'Govt Official (Appointed/Elected)', 'nature_of_work_new'] = df.loc[df['nature_of_work'] == 'Govt Official (Appointed/Elected)'].apply(lambda x: assign_govt_official(), axis=1)

# Display the DataFrame
df.groupby(['nature_of_work', 'nature_of_work_new'])['digitalLoanAccountId'].nunique()

nature_of_work                                               nature_of_work_new                                         
Accountant/Auditor                                           Licensed Professional - Accountant                              19005
Admin/Clerical Staff/Officer                                 missing                                                         56980
Architect/Interior Decorator                                 Licensed Professional - Others                                   2855
Bank Employee                                                missing                                                         14844
Broker/Dealer                                                Broker/Dealer                                                    2485
Call Center Agent/Tele Marketer                              missing                                                        115422
Cashier/Food Server/Waiter                                   missing                         

In [6]:
df['nature_of_work_new'].nunique()

12

In [7]:
# # Convert the 'nature_of_work' column to uppercase
# df['nature_of_work'] = df['nature_of_work'].str.upper()

# # Mapping dictionary (keys and values in uppercase)
# mapping = {
#     'ADMIN/CLERICAL STAFF/OFFICER': 'missing',
#     'ARCHITECT/INTERIOR DECORATOR': 'LICENSED PROFESSIONAL - OTHERS',
#     'BANK EMPLOYEE': 'missing',
#     'BROKER/DEALER': 'BROKER/DEALER',
#     'CALL CENTER AGENT/TELE MARKETER': 'missing',
#     'CASHIER/FOOD SERVER/WAITER': 'missing',
#     'CONSULTANT': 'missing',
#     'DOCTOR/DENTIST/MEDICAL PROFESSIONAL': 'LICENSED PROFESSIONAL - OTHERS',
#     'ENGINEER': 'LICENSED PROFESSIONAL - OTHERS',
#     'ENTERTAINER/ARTIST': 'missing',
#     'FOREIGN AMBASSADOR/DIPLOMAT': 'missing',
#     'GOVT EMPLOYEE': 'GOVT EMPLOYEE',
#     'INSURANCE AGENT/FINANCIAL ADVISOR': 'missing',
#     'IT PROFESSIONAL': 'missing',
#     'LAWYER': 'LICENSED PROFESSIONAL - LAWYER',
#     'MILITARY/ARMED FORCES/POLICE/NAVY/AIRFORCE': 'GOVERNMENT - UNIFORMED SERVICE',
#     'NEWSCASTER/MEDIA PERSONALITY': 'missing',
#     'PILOT/AIRLINE CREW': 'missing',
#     'SEAFARER': 'missing',
#     'TEACHER/INSTRUCTOR/COACH': 'missing',
#     'UTILITY PERSONNEL/HOUSEHOLD HELP': 'missing',
#     'ACCOUNTANT/AUDITOR': 'LICENSED PROFESSIONAL - ACCOUNTANT ',
#     'UTILITIES (ELECTRICITY, GAS AND WATER)': 'missing',
#     'WHOLESALE AND RETAIL TRADE': 'missing',
#     'SALES/MARKETING PERSONNEL': 'SALES/MARKETING PERSONNEL',
#     'OTHER PROFESSIONAL SERVICES': 'LICENSED PROFESSIONAL - OTHERS',
#     'OTHER NON PROFESSIONAL SERVICES': 'missing',
#     'OWNER': 'OWNER',
#     'RELIGIOUS LEADER': 'RELIGIOUS LEADER',
#     'FIRE FIGHTER': 'GOVERNMENT - UNIFORMED SERVICE',
#     'FARMER/FISHERMAN': 'missing',
#     'DEBT COLLECTION/RECOVERY AGENTS/ COLLECTION AGENCY EMPLOYEE': 'DEBT COLLECTION/RECOVERY AGENTS/ COLLECTION AGENCY EMPLOYEE',
#     'COURT PERSONNEL/BAILIFF': 'COURT PERSONNEL/BAILIFF',
#     'STORE/SERVICE MANAGER': 'MANAGER/EXECUTIVE',
#     'SALES MARKETING PROFESSIONAL': 'SALES/MARKETING PERSONNEL',
#     'BARBER / BEAUTY SALON': 'SALES/MARKETING PERSONNEL',
#     'BUSINESS OWNER (MSME)': 'OWNER',
#     'DELIVERY RIDER/DRIVER': 'DELIVERY RIDER/DRIVER',
#     'MANPOWER AGENCY': 'missing',
#     'ONLINE FREELANCING / VIRTUAL ASSISTANT / ONLINE TEACHER': 'VIRTUAL/ONLINE/REMOTE EMPLOYEE',
#     'ONLINE SELLER': 'VIRTUAL/ONLINE/REMOTE EMPLOYEE',
#     'REAL ESTATE AGENT': 'missing',
#     'SKILLED WORKER': 'SKILLED WORKER',
#     'TECHNICIAN (CELLPHONE / LAPTOP)': 'missing',
#     'VENDOR (SARI-SARI STORE, WET MARKET, ETC)': 'missing',
#     'STAFF/RANK AND FILE': 'STAFF/RANK AND FILE',
#     'OFFICER/SUPERVISOR ': 'OFFICER/SUPERVISOR ',
#     'MANAGER/EXECUTIVE': 'MANAGER/EXECUTIVE',
#     'LICENSED PROFESSIONAL - LAWYER': 'LICENSED PROFESSIONAL - LAWYER',
#     'LICENSED PROFESSIONAL - ACCOUNTANT ': 'LICENSED PROFESSIONAL - ACCOUNTANT ',
#     'LICENSED PROFESSIONAL - OTHERS': 'LICENSED PROFESSIONAL - OTHERS',
#     'RELIGIOUS MINISTER ': 'RELIGIOUS MINISTER ',
#     'GOVERNMENT - ELECTED OFFICIAL ': 'GOVERNMENT - ELECTED OFFICIAL ',
#     'GOVERNMENT - UNIFORMED SERVICE': 'GOVERNMENT - UNIFORMED SERVICE',
#     'DIRECTOR/TRUSTEE/STOCKHOLDER': 'DIRECTOR/TRUSTEE/STOCKHOLDER',
#     'VIRTUAL/ONLINE/REMOTE EMPLOYEE': 'VIRTUAL/ONLINE/REMOTE EMPLOYEE',
#     'GOVERNMENT - APPOINTED OFFICIAL': 'GOVERNMENT - APPOINTED OFFICIAL'
# }

# # Apply the mapping
# df['nature_of_work_new'] = df['nature_of_work'].map(mapping)

# # Handle the special case for 'GOVT OFFICIAL (APPOINTED/ELECTED)'
# def assign_govt_official():
#     return np.random.choice(
#         ['GOVERNMENT - APPOINTED OFFICIAL', 'GOVERNMENT - ELECTED OFFICIAL'],
#         p=[0.9, 0.1]
#     )

# df.loc[df['nature_of_work'] == 'GOVT OFFICIAL (APPOINTED/ELECTED)', 'nature_of_work_new'] = df.loc[df['nature_of_work'] == 'GOVT OFFICIAL (APPOINTED/ELECTED)'].apply(lambda x: assign_govt_official(), axis=1)

# # Display the DataFrame
# df.sample(20)

In [8]:
df[df['nature_of_work']=='GOVT OFFICIAL (APPOINTED/ELECTED)'].head()

,digitalLoanAccountId,nature_of_work,industry_description,employment_type,source_funds,nature_of_work_new


In [9]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class NatureOfWorkTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # Define the mapping dictionary
        self.mapping = {
            'Admin/Clerical Staff/Officer': 'missing',
            'Architect/Interior Decorator': 'Licensed Professional - Others',
            'Bank Employee': 'missing',
            'Broker/Dealer': 'Broker/Dealer',
            'Call Center Agent/Tele Marketer': 'missing',
            'Cashier/Food Server/Waiter': 'missing',
            'Consultant': 'missing',
            'Doctor/Dentist/Medical Professional': 'Licensed Professional - Others',
            'Engineer': 'Licensed Professional - Others',
            'Entertainer/Artist': 'missing',
            'Foreign Ambassador/Diplomat': 'missing',
            'Govt Employee': 'Govt Employee',
            'Insurance Agent/Financial Advisor': 'missing',
            'IT Professional': 'missing',
            'Lawyer': 'Licensed Professional - Lawyer',
            'Military/Armed Forces/Police/Navy/Airforce': 'Government - Uniformed Service',
            'Newscaster/Media Personality': 'missing',
            'Pilot/Airline Crew': 'missing',
            'Seafarer': 'missing',
            'Teacher/Instructor/Coach': 'missing',
            'Utility Personnel/Household Help': 'missing',
            'Accountant/Auditor': 'Licensed Professional - Accountant ',
            'Utilities (Electricity, Gas and Water)': 'missing',
            'Wholesale and Retail Trade': 'missing',
            'Sales/Marketing Personnel': 'Sales/Marketing Personnel',
            'Other Professional services': 'Licensed Professional - Others',
            'Other Non Professional Services': 'missing',
            'Owner': 'Owner',
            'Religious Leader': 'Religious Leader',
            'Fire Fighter': 'Government - Uniformed Service',
            'Farmer/Fisherman': 'missing',
            'Debt Collection/Recovery Agents/ Collection Agency Employee': 'Debt Collection/Recovery Agents/ Collection Agency Employee',
            'Court Personnel/Bailiff': 'Court Personnel/Bailiff',
            'Store/Service Manager': 'Manager/Executive',
            'Sales Marketing Professional': 'Sales/Marketing Personnel',
            'Barber / Beauty Salon': 'Sales/Marketing Personnel',
            'Business Owner (MSME)': 'Owner',
            'Delivery Rider/Driver': 'Delivery Rider/Driver',
            'Manpower Agency': 'missing',
            'Online Freelancing / Virtual Assistant / Online teacher': 'Virtual/Online/Remote Employee',
            'Online Seller': 'Virtual/Online/Remote Employee',
            'Real Estate agent': 'missing',
            'Skilled Worker': 'Skilled Worker',
            'Technician (Cellphone / Laptop)': 'missing',
            'Vendor (Sari-Sari store, Wet market, etc)': 'missing',
            'Staff/Rank and File': 'Staff/Rank and File',
            'Officer/Supervisor ': 'Officer/Supervisor ',
            'Manager/Executive': 'Manager/Executive',
            'Licensed Professional - Lawyer': 'Licensed Professional - Lawyer',
            'Licensed Professional - Accountant ': 'Licensed Professional - Accountant ',
            'Licensed Professional - Others': 'Licensed Professional - Others',
            'Religious Minister ': 'Religious Minister ',
            'Government - Elected Official ': 'Government - Elected Official ',
            'Government - Uniformed Service': 'Government - Uniformed Service',
            'Director/Trustee/Stockholder': 'Director/Trustee/Stockholder',
            'Virtual/Online/Remote Employee': 'Virtual/Online/Remote Employee',
            'Government - Appointed Official': 'Government - Appointed Official'
        }
        self.output_format = "default"  # Default output format
        self.feature_names = None  # Placeholder for feature names

    def fit(self, X, y=None):
        # No fitting required for this transformer
        return self

    def transform(self, X):
        # Ensure X is a DataFrame
        X = X.copy()

        # Apply the mapping
        X['nature_of_work_new'] = X['nature_of_work'].map(self.mapping)

        # Handle the special case for 'Govt Official (Appointed/Elected)'
        def assign_govt_official():
            return np.random.choice(
                ['Government - Appointed Official', 'Government - Elected Official'],
                p=[0.9, 0.1]
            )

        X.loc[X['nature_of_work'] == 'Govt Official (Appointed/Elected)', 'nature_of_work_new'] = X.loc[
            X['nature_of_work'] == 'Govt Official (Appointed/Elected)'
        ].apply(lambda x: assign_govt_official(), axis=1)

        # Store feature names for get_feature_names_out
        self.feature_names = X.columns.tolist()

        # Set output format
        if self.output_format == "pandas":
            return X
        else:
            return X.values  # Return as NumPy array by default

    def get_feature_names_out(self, input_features=None):
        """
        Get the output feature names for transformation.
        """
        if self.feature_names is None:
            raise ValueError("Transformer has not been fitted yet.")
        return self.feature_names

    def set_output(self, transform="default"):
        """
        Set the output format of the transformer.
        Options: "default" (NumPy array) or "pandas" (Pandas DataFrame).
        """
        if transform not in ["default", "pandas"]:
            raise ValueError("Output format must be 'default' or 'pandas'.")
        self.output_format = transform
        return self

In [10]:
# Example how to use it

# Sample DataFrame
data = {
    'nature_of_work': [
        'Admin/Clerical Staff/Officer', 'Architect/Interior Decorator', 'Bank Employee',
        'Govt Official (Appointed/Elected)', 'Lawyer', 'Military/Armed Forces/Police/Navy/Airforce'
    ]
}
dfd = pd.DataFrame(data)

# Initialize the transformer
transformer = NatureOfWorkTransformer()

# Apply the transformer to the DataFrame
df_transformed = transformer.transform(dfd)

# Display the transformed DataFrame
print(df_transformed)

[['Admin/Clerical Staff/Officer' 'missing']
 ['Architect/Interior Decorator' 'Licensed Professional - Others']
 ['Bank Employee' 'missing']
 ['Govt Official (Appointed/Elected)' 'Government - Appointed Official']
 ['Lawyer' 'Licensed Professional - Lawyer']
 ['Military/Armed Forces/Police/Navy/Airforce'
  'Government - Uniformed Service']]


In [11]:
df[['digitalLoanAccountId', 'nature_of_work', 'nature_of_work_new']].to_csv("Nature_of_work_mapping.csv", index = False)

# industry_description

In [12]:
# Mapping dictionary
mapping = {
    'Accounting/Bookkeeping': 'Accounting/Bookkeeping/Auditing',
    'Advertising/Sales/Marketing': 'Advertising/Sales/Marketing',
    'Agribusiness/Animal Farming/Fishing': 'Agribusiness/Animal Farming/Fishing',
    'Airline/Aviation': 'Airline/Aviation',
    'Architecture/Engineering': 'Architecture/Engineering',
    'Banking': 'Banking',
    'Beauty and Health Services': 'Beauty and Health Services',
    'Business Process Outsourcing/Tele Marketing': 'Business Process Outsourcing/Tele Marketing',
    'Car/Boat/Plane/Used Truck/AutoParts Manufacturing/Dealership or Auction Houses': 'Car/Boat/Plane/Used Truck/AutoParts Manufacturing/Dealership or Auction Houses',
    'Computer/IT Related Services': 'Computer/IT Related Services',
    'Construction': 'Construction',
    "Convenience Store/Internet Cafe's": "Convenience Store/Internet Cafe's",
    'Designated NonFinancial Businesses': 'Designated NonFinancial Businesses',
    'Education/Schools': 'Education/Private Schools',
    '': '',
    'Fin Tech/Financial Services': 'Fin Tech/Financial Services',
    'Food Industry/Restaurants/Fastfoods': 'Food Industry/Restaurants/Fastfoods',
    'Freight and Cargo': 'Freight and Cargo',
    'FX Dealer/Money Changer': 'FX Dealer/Money Changer',
    'Online Gaming/Lottery/Casino': 'Gambling and Betting (Online Gaming, Lottery, Casino)',
    'Gold, Jewelry and Precious Metals': 'Gold/Jewelry/Precious Metals/Art/Antique Dealership',
    'Govt. Institution': 'Govt. Institution',
    'Hospital and Medical Services': 'Hospital and Medical Services',
    'Repair, Sanitation and Housekeeping': 'Housekeeping/Sanitation Services',
    'Import Export Companies': 'Import Export Companies',
    'Insurance Activities': 'Insurance Activities',
    'Legal Service/Law Firm': 'Legal Service/Law Firm',
    'Manpower/Manning Agencies': 'Manpower/Manning Agencies',
    'Manufacturing - Food': 'Manufacturing',
    'Manufacturing - Non Food': 'Manufacturing',
    'Maritime/Shipping': 'Maritime/Shipping',
    'Military and Weaponry Business': 'Military and Weaponry Business',
    '': 'Mining and Quarrying',
    'Non Government Organization/Foundation': 'NGOs/NonProfit Organization/Foundation',
    'NGOs/NonProfit Organization/Foundation': 'NGOs/NonProfit Organization/Foundation',
    'Nightclubs/Discos/Karaoke Center': 'Nightclubs/Discos/Karaoke Center',
    'Others': 'Others',
    'Pawnshop/Remittance Agent/Virtual Currency': 'Pawnshop/Remittance Agent/Virtual Currency',
    'Real Estate': 'Real Estate',
    'Entertainment and Media': 'Recreation (Arts, Entertainment/Media, Sports)',
    'Religious Organization': 'Religious Organization',
    '': 'Securities/Holding Companies',
    'Telecommunication': 'Telecommunication',
    'Tourism, Hotel and Resorts': 'Tourism, Hotel and Resorts',
    'Transportation': 'Transportation',
    'Utilities (Electricity, Gas and Water)': 'Utilities (Electricity, Gas and Water)',
    'Wholesale and Retail Trade': 'Wholesale and Retail Trade',
    'Publishing and Printing': 'missing',
    'Recycling': 'missing',
    'Embassy': 'missing',
    'Leasing and Rental Services': 'missing'
}

# Apply the mapping to create the new column
df['industry_description_new'] = df['industry_description'].map(mapping)

# Display the DataFrame
df.groupby(['industry_description', 'industry_description_new'])['digitalLoanAccountId'].nunique()

industry_description                         industry_description_new                             
Accounting/Bookkeeping                       Accounting/Bookkeeping/Auditing                           12421
Advertising/Sales/Marketing                  Advertising/Sales/Marketing                               27861
Agribusiness/Animal Farming/Fishing          Agribusiness/Animal Farming/Fishing                       11956
Airline/Aviation                             Airline/Aviation                                           1864
Architecture/Engineering                     Architecture/Engineering                                   6873
Banking                                      Banking                                                   19428
Beauty and Health Services                   Beauty and Health Services                                15513
Business Process Outsourcing/Tele Marketing  Business Process Outsourcing/Tele Marketing              115421
Computer/IT Related Services 

In [13]:
df.head()

,digitalLoanAccountId,nature_of_work,industry_description,employment_type,source_funds,nature_of_work_new,industry_description_new
0,2da28348-214c-4a16-ab18-3aa3212103e5,Debt Collection/Recovery Agents/ Collection Ag...,Others,Employed,Salary,Debt Collection/Recovery Agents/ Collection Ag...,Others
1,27ebbaa8-8e29-4290-bff8-094e3c36e2a3,Pilot/Airline Crew,Airline/Aviation,Employed,Salary,missing,Airline/Aviation
2,3cd55c88-1734-4f83-a98f-3657e88dfa06,Broker/Dealer,Others,Employed,Salary,Broker/Dealer,Others
3,a404dc36-481a-401d-a14b-509d35022896,Lawyer,"Utilities (Electricity, Gas and Water)",Employed,Salary,Licensed Professional - Lawyer,"Utilities (Electricity, Gas and Water)"
4,7d45e1cd-a7cb-45cd-9400-1b5956be1b40,Lawyer,"Utilities (Electricity, Gas and Water)",Employed,Salary,Licensed Professional - Lawyer,"Utilities (Electricity, Gas and Water)"


In [14]:
df[['digitalLoanAccountId', 'industry_description', 'industry_description_new']].to_csv("Natureofwork_industry_descriptionMappingwithnewlov.csv", index = False)

# source_funds

In [15]:
# Mapping dictionary
mapping = {
    'Salary': 'Salary ',
    'Allowance': 'Allowance ',
    'Commissions': 'missing',  # Make it empty or null
    'Income from Business': 'Income from Business ',
    'Inheritance/Retirement Funds': 'Inheritance/Retirement Funds',
    'Interest on Deposits and Investments': 'Interest on Deposits and Investments',
    'Pension': 'Pension ',
    'Remittance': 'Remittance ',
    'Others': 'missing',
    'Donations': 'Donations '
}

# Apply the mapping to create the new column
df['source_funds_new'] = df['source_funds'].map(mapping)
# Display the DataFrame
df.groupby(['source_funds', 'source_funds_new'])['digitalLoanAccountId'].nunique()

source_funds                          source_funds_new                    
Allowance                             Allowance                                58079
Commissions                           missing                                  12575
Income from Business                  Income from Business                    136341
Inheritance/Retirement Funds          Inheritance/Retirement Funds              1487
Interest on Deposits and Investments  Interest on Deposits and Investments      1579
Others                                missing                                  35947
Pension                               Pension                                   2216
Remittance                            Remittance                               66807
Salary                                Salary                                  702252
Name: digitalLoanAccountId, dtype: int64

In [16]:
df[['digitalLoanAccountId', 'source_funds', 'source_funds_new']].to_csv(r"Source_fund_mapping.csv", index = False)

# employment_type

In [17]:
# Mapping dictionary
mapping = {
    'Employed': 'Employed',  # Placeholder, will be handled separately
    'Self-Employed/Private Practice': 'Self-Employed/Private Practice',
    'Business Owner': 'Business Owner',
    'Overseas Filipino Worker': 'Overseas Filipino Worker',
    'Unemployed': 'Unemployed',
    'Retired - Private Employee': 'Retired - Private Employee',
    'Retired Govt. Employee/ Govt. Official': 'Retired Govt. Employee/ Govt. Official',
    'Housewife/Househusband': 'Housewife/Househusband',
    'Student': 'Student',
    'Remittance Beneficiary': 'Remittance Beneficiary',
    'Freelancer/Independent Contractor ': 'Freelancer/Independent Contractor ',
    'Religious ': 'Religious '
}

# Apply the mapping to create the new column
df['employment_type_new'] = df['employment_type'].map(mapping)

# Handle the special case for 'Employed'
def assign_employed():
    return np.random.choice(
        ['Employed - Private Employee', 'Employed - Govt. Employee/Govt. Official'],
        p=[0.7, 0.3]
    )

df.loc[df['employment_type'] == 'Employed', 'employment_type_new'] = df.loc[
    df['employment_type'] == 'Employed'
].apply(lambda x: assign_employed(), axis=1)

# Display the DataFrame
df.groupby(['employment_type', 'employment_type_new'])['digitalLoanAccountId'].nunique()

employment_type                 employment_type_new                     
Business Owner                  Business Owner                               31554
Employed                        Employed - Govt. Employee/Govt. Official    187072
                                Employed - Private Employee                 435954
Housewife/Househusband          Housewife/Househusband                       17284
Overseas Filipino Worker        Overseas Filipino Worker                    124472
Remittance Beneficiary          Remittance Beneficiary                       48092
Retired - Private Employee      Retired - Private Employee                   32674
Self-Employed/Private Practice  Self-Employed/Private Practice              110028
Student                         Student                                       2569
Unemployed                      Unemployed                                   22926
Name: digitalLoanAccountId, dtype: int64

In [18]:
df[['digitalLoanAccountId','employment_type', 'employment_type_new']].to_csv("Employment_type_mapping.csv", index = False)

In [19]:
df.to_csv("Now_ID_SF_ET_mapping.csv", index = False)

In [20]:
def save_df_to_gcs(df, bucket_name, destination_blob_name, file_format='csv'):
    """Saves a pandas DataFrame to Google Cloud Storage.

    Args:
        df: The pandas DataFrame to save.
        bucket_name: The name of the GCS bucket.
        destination_blob_name: The name of the blob to be created.
        file_format: The file format to save the DataFrame in ('csv' or 'parquet').
    """

    # Create a temporary file
    if file_format == 'csv':
        temp_file = 'temp.csv'
        df.to_csv(temp_file, index=False)
    elif file_format == 'parquet':
        temp_file = 'temp.parquet'
        df.to_parquet(temp_file, index=False)
    else:
        raise ValueError("Invalid file format. Please choose 'csv' or 'parquet'.")

    # Upload the file to GCS
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(temp_file)

    # Remove the temporary file
    import os
    os.remove(temp_file)
    


In [21]:
from datetime import datetime
current_date = datetime.now().strftime("%Y%m%d")
BUCKET_NAME = "prod-asia-southeast1-tonik-aiml-workspace"
filename = 'Now_ID_SF_ET_mapping'
CLOUDPATH = "Monthly_Income_Estimation"
bucket_name = BUCKET_NAME
new_filename = f"{current_date}_{filename}.csv"
print(new_filename)
destination_blob_name = f"{CLOUDPATH}/{new_filename}"
save_df_to_gcs(df, bucket_name, destination_blob_name)

20250122_Now_ID_SF_ET_mapping.csv
